In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json

# read all csv
def read_and_process_file(filename, pollutant):
    df = pd.read_csv(filename)
    # setting the same date-format 
    df['Date'] = pd.to_datetime(df['Date'])
    if pollutant == 'CO':
        value_col = 'Daily Max 8-hour CO Concentration'
    elif pollutant == 'NO2':
        value_col = 'Daily Max 1-hour NO2 Concentration'
    elif pollutant == 'O3':
        value_col = 'Daily Max 8-hour Ozone Concentration'
    elif pollutant == 'PM2.5':
        value_col = 'Daily Mean PM2.5 Concentration'
    elif pollutant == 'PM10':
        value_col = 'Daily Mean PM10 Concentration'
    elif pollutant == 'SO2':
        value_col = 'Daily Max 1-hour SO2 Concentration'
    
    selected_cols = ['Date', 'Site ID', 'Local Site Name', 
                    'Site Latitude', 'Site Longitude', value_col, 
                    'Units', 'Daily AQI Value']
    
    df = df[selected_cols]
    # rename the concentration columns
    df = df.rename(columns={value_col: 'Concentration'})

    df['Pollutant'] = pollutant
    return df

# read all csv
files = {
    'CO': 'CO.csv',
    'NO2': 'NO2.csv',
    'O3': 'ozone.csv',
    'PM2.5': 'PM2.5.csv',
    'PM10': 'PM10.csv',
    'SO2': 'SO2.csv'
}

# combine the data
dfs = []
for pollutant, filename in files.items():
    df = read_and_process_file(filename, pollutant)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

# check for each location pollution number
station_coverage = combined_df.groupby('Site ID')['Pollutant'].nunique()
print("site ID pollutaion air number：")
print(station_coverage)



site ID pollutaion air number：
Site ID
60370016    5
60370113    2
60371103    6
60371201    4
60371302    4
60371602    4
60371701    3
60372005    4
60374006    1
60374008    2
60374009    5
60374010    3
60376012    5
60379035    4
Name: Pollutant, dtype: int64


In [2]:
def create_data_for_visualization():

    base_data = combined_df.copy()

    base_data['Date'] = base_data['Date'].dt.strftime('%Y-%m-%d')
    
    # each location stats calculation
    station_stats = base_data.groupby(['Site ID', 'Local Site Name', 'Site Latitude', 'Site Longitude', 'Pollutant']).agg({
        'Concentration': ['count', 'mean', 'max', 'min'],
        'Daily AQI Value': ['mean', 'max', 'min']
    }).reset_index()
    
    # rename the columns
    station_stats.columns = [
        'Site ID', 'Local Site Name', 'Site Latitude', 'Site Longitude', 'Pollutant',
        'Concentration_count', 'Concentration_mean', 'Concentration_max', 'Concentration_min',
        'AQI_mean', 'AQI_max', 'AQI_min'
    ]
    
    # each pollutant stats calculation
    pollutant_stats = base_data.groupby(['Pollutant', 'Units']).agg({
        'Concentration': ['count', 'mean', 'max', 'min'],
        'Daily AQI Value': ['mean', 'max', 'min']
    }).reset_index()
    
    # rename the columns
    pollutant_stats.columns = [
        'Pollutant', 'Units',
        'Concentration_count', 'Concentration_mean', 'Concentration_max', 'Concentration_min',
        'AQI_mean', 'AQI_max', 'AQI_min'
    ]
    
    # prepare for the data
    output_data = {
        # time series
        'timeSeriesData': base_data[['Date', 'Site ID', 'Pollutant', 'Concentration', 'Daily AQI Value', 'Units']].to_dict('records'),
        
        # use for the map 
        'stations': base_data.groupby(['Site ID', 'Local Site Name', 'Site Latitude', 'Site Longitude']).agg({
            'Pollutant': lambda x: sorted(list(set(x)))
        }).reset_index().to_dict('records'),
        
        # for pie and bar chart 
        'stationStats': station_stats.to_dict('records'),
        
        'pollutantStats': pollutant_stats.to_dict('records'),
        
        # for drop down tools
        'metadata': {
            'dateRange': {
                'start': base_data['Date'].min(),
                'end': base_data['Date'].max()
            },
            'pollutants': sorted(list(base_data['Pollutant'].unique())),
            'stationNames': sorted(list(base_data['Local Site Name'].unique())),
            'units': base_data.groupby('Pollutant')['Units'].first().to_dict()
        }
    }
    
    return output_data

# data generation
output_data = create_data_for_visualization()

# json file
with open('air_quality_visualization.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

In [3]:
print("\ndata overlook：")
print(f"timeSeriesData：{len(output_data['timeSeriesData'])}")
print(f"stations：{len(output_data['stations'])}")
print(f"stationStats：{len(output_data['stationStats'])}")
print(f"pollutantStats：{len(output_data['pollutantStats'])}")
print(f"pollutants：{len(output_data['metadata']['pollutants'])}")

print("\ntimeSeriesData(demo)：")
print(json.dumps(output_data['timeSeriesData'][0], indent=2))

print("\nstations（demo）：")
print(json.dumps(output_data['stations'][0], indent=2))

print("\nstationStats（demo）：")
print(json.dumps(output_data['stationStats'][0], indent=2))

print("\npollutantStats（demo）：")
print(json.dumps(output_data['pollutantStats'][0], indent=2))


data overlook：
timeSeriesData：14563
stations：14
stationStats：52
pollutantStats：6
pollutants：6

timeSeriesData(demo)：
{
  "Date": "2024-01-01",
  "Site ID": 60370016,
  "Pollutant": "CO",
  "Concentration": 0.2,
  "Daily AQI Value": 2,
  "Units": "ppm"
}

stations（demo）：
{
  "Site ID": 60370016,
  "Local Site Name": "Glendora",
  "Site Latitude": 34.14435,
  "Site Longitude": -117.85036,
  "Pollutant": [
    "CO",
    "NO2",
    "O3",
    "PM10",
    "PM2.5"
  ]
}

stationStats（demo）：
{
  "Site ID": 60370016,
  "Local Site Name": "Glendora",
  "Site Latitude": 34.14435,
  "Site Longitude": -117.85036,
  "Pollutant": "CO",
  "Concentration_count": 222,
  "Concentration_mean": 0.31216216216216214,
  "Concentration_max": 0.5,
  "Concentration_min": 0.2,
  "AQI_mean": 3.445945945945946,
  "AQI_max": 6,
  "AQI_min": 2
}

pollutantStats（demo）：
{
  "Pollutant": "CO",
  "Units": "ppm",
  "Concentration_count": 1922,
  "Concentration_mean": 0.38636836628511967,
  "Concentration_max": 2.1,
  "Co